In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import optimizers
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling1D
from keras import regularizers, optimizers

In [ ]:
im = cv2.imread('../input/happy-whale-and-dolphin/train_images/002618d6f63ebc.jpg')
im.shape

In [ ]:
train_df = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
train_df.head()

In [ ]:
test_ids = os.listdir('../input/happy-whale-and-dolphin/test_images')
test_df = pd.DataFrame({'image_ids':test_ids})
test_df.head()

In [ ]:
train_df['individual_id'].value_counts()

In [ ]:
len(pd.unique(train_df['individual_id']))

In [ ]:
train_dir = "../input/happy-whale-and-dolphin/train_images"

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   height_shift_range = 0.1,
                                   width_shift_range = 0.1,
                                validation_split = 0.2,
                                horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

train_set = train_datagen.flow_from_dataframe(train_df, train_dir,
                                              seed = 101,
                                            target_size = (240,240),
                                            batch_size = 12,
                                            x_col='image',
                                            y_col='individual_id',
                                            class_mode = 'categorical',
                                            subset = 'training')

valid_set = valid_datagen.flow_from_dataframe(train_df, 
                                              train_dir,
                                                seed = 101,
                                                target_size = (240,240),
                                              x_col='image',
                                                y_col='individual_id',
                                                batch_size = 12,
                                                class_mode = 'categorical',
                                                subset = 'validation')

  


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_dir = '../input/happy-whale-and-dolphin/test_images'

test_set = test_datagen.flow_from_dataframe(test_df,
                                            test_dir,
                                            x_col='image_ids',
                                            y_col=None,
                                            seed = 101,
                                            batch_size = 12,
                                            class_mode=None,
                                            target_size = (240,240)                           
)

# CNN Model

In [ ]:
model  = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(240,240,3),
    pooling=None,
    classes=15587,
    classifier_activation="softmax",
)

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(15587, activation='softmax')(class1)
model = keras.Model(inputs=model.inputs, outputs=output)

model.summary()

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=[tf.keras.metrics.Precision(top_k=5)])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True)
]

In [ ]:
STEP_SIZE_TRAIN=train_set.n//train_set.batch_size
STEP_SIZE_VALID=valid_set.n//valid_set.batch_size
STEP_SIZE_TEST=test_set.n//test_set.batch_size
model.fit_generator(generator=train_set,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)